### Load the Dataset

In [68]:
import pandas as pd
import numpy as np
import streamlit as st
import re

In [69]:
df = pd.read_csv('Mobiles_Dataset.csv')
# df = pd.read_csv('Cleaned_Mobiles_Dataset.csv')

### Explore the dataset

In [70]:
df.head()

,Product Name,Actual price,Discount price,Stars,Rating,Reviews,RAM (GB),Storage (GB),Display Size (inch),Camera,Description,Link
0,"Apple iPhone 15 (Green, 128 GB)","₹79,600","₹65,999",4.6,"44,793 Ratings","2,402 Reviews",NIL,128,6.10,48MP + 12MP,128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...,https://www.flipkart.com/apple-iphone-15-green...
1,"Apple iPhone 15 (Blue, 128 GB)","₹79,600","₹65,999",4.6,"44,793 Ratings","2,402 Reviews",NIL,128,6.10,48MP + 12MP,128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...,https://www.flipkart.com/apple-iphone-15-blue-...
2,"Apple iPhone 15 (Black, 128 GB)","₹79,600","₹65,999",4.6,"44,793 Ratings","2,402 Reviews",NIL,128,6.10,48MP + 12MP,128 GB ROM15.49 cm (6.1 inch) Super Retina XDR...,https://www.flipkart.com/apple-iphone-15-black...
3,"OnePlus N20 SE (JADE WAVE, 128 GB)","₹19,999","₹11,489",4.0,"1,005 Ratings",41 Reviews,4,128,6.56,50MP,4 GB RAM | 128 GB ROM16.66 cm (6.56 inch) Disp...,https://www.flipkart.com/oneplus-n20-se-jade-w...
4,"OnePlus N20 SE (BLUE OASIS, 64 GB)","₹16,999","₹12,999",4.0,"1,005 Ratings",41 Reviews,4,64,6.56,50MP,4 GB RAM | 64 GB ROM16.66 cm (6.56 inch) Displ...,https://www.flipkart.com/oneplus-n20-se-blue-o...


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Product Name         984 non-null    object 
 1   Actual price         984 non-null    object 
 2   Discount price       984 non-null    object 
 3   Stars                984 non-null    float64
 4   Rating               984 non-null    object 
 5   Reviews              984 non-null    object 
 6   RAM (GB)             984 non-null    object 
 7   Storage (GB)         984 non-null    object 
 8   Display Size (inch)  984 non-null    float64
 9   Camera               908 non-null    object 
 10  Description          984 non-null    object 
 11  Link                 984 non-null    object 
dtypes: float64(2), object(10)
memory usage: 92.4+ KB


### Preprocess the Dataset

In [71]:
#change the name of the branch "I KALL" to "IKALL" for better brand extraction
df['Product Name'] = df['Product Name'].str.replace('I kall', 'IKall', case=False)

#extract the brandnames and then save it another column
df['Brand'] = df['Product Name'].str.split().str[0]
df['Brand'] = df['Brand'].str.lower()

#drop the decription and links column
df = df.drop('Description', axis=1)
df = df.drop('Link', axis=1)

In [72]:
# Clean 'Actual price' column
df['Actual price'] = (
    df['Actual price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)
df['Actual price'] = pd.to_numeric(df['Actual price'], errors='coerce')

# Clean 'Discount price' column
df['Discount price'] = (
    df['Discount price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
)
df['Discount price'] = pd.to_numeric(df['Discount price'], errors='coerce')

In [73]:
#drop rows with missing prices
df.dropna(subset=['Actual price', 'Discount price'], inplace=True)

# Convert to integer
df['Actual price'] = df['Actual price'].astype(int)
df['Discount price'] = df['Discount price'].astype(int)


In [74]:
#fix ram and storage columns
def extract_ram(ram_str):
    if pd.isna(ram_str):
        return None
    match = re.search(r'(\d+)', ram_str)
    return int(match.group(1)) if match else None
def extract_storage(storage_str):
    if pd.isna(storage_str):
        return None
    match = re.search(r'(\d+)', storage_str)
    return int(match.group(1)) if match else None

df['RAM (GB)'] = df['RAM (GB)'].apply(extract_ram).astype("Int64")
df['Storage (GB)'] = df['Storage (GB)'].apply(extract_storage).astype("Int64")


In [75]:
#cleaning the reviws and rating columns
df['Rating'] = df['Rating'].str.replace(' Ratings', '').str.replace(',', '').astype(int)
df['Reviews'] = df['Reviews'].str.replace(' Reviews', '').str.replace(',', '').astype(int)

In [76]:
#workaround to extract primary camera lens from the camera column
def extract_primary_camera(mp_str):
    if pd.isna(mp_str):
        return None
    match = re.search(r'(\d+)', mp_str) #search for numbers
    return int(match.group(1)) if match else None #return the first number if found else none

df['Primary Camera (MP)'] = df['Camera'].apply(extract_primary_camera)

#drop the camera column
df = df.drop('Camera', axis=1)

In [77]:
df.head()
# df.tail()

,Product Name,Actual price,Discount price,Stars,Rating,Reviews,RAM (GB),Storage (GB),Display Size (inch),Brand,Primary Camera (MP)
0,"Apple iPhone 15 (Green, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0
1,"Apple iPhone 15 (Blue, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0
2,"Apple iPhone 15 (Black, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0
3,"OnePlus N20 SE (JADE WAVE, 128 GB)",19999,11489,4.0,1005,41,4,128,6.56,oneplus,50.0
4,"OnePlus N20 SE (BLUE OASIS, 64 GB)",16999,12999,4.0,1005,41,4,64,6.56,oneplus,50.0


In [78]:
df.describe()


,Actual price,Discount price,Stars,Rating,Reviews,RAM (GB),Storage (GB),Display Size (inch),Primary Camera (MP)
count,930.000000,930.000000,930.000000,930.000000,930.000000,774.0,826.0,930.000000,857.000000
mean,25263.134409,20023.831183,4.249892,22843.284946,1450.666667,128.608527,179.228814,6.019953,40.693116
std,22257.240721,18090.048296,0.195385,51729.880364,2889.385691,2380.869118,103.619407,1.605950,28.497423
min,1199.000000,809.000000,3.400000,4.000000,0.000000,2.0,0.0,0.660000,0.000000
25%,11999.000000,8706.750000,4.200000,697.000000,42.250000,6.0,128.0,6.500000,12.000000
50%,19999.000000,15858.500000,4.200000,5560.000000,346.000000,8.0,128.0,6.670000,50.000000
75%,30999.000000,25826.500000,4.400000,17356.000000,1414.000000,8.0,256.0,6.720000,50.000000
max,149999.000000,129999.000000,4.700000,429459.000000,23258.000000,46875.0,512.0,7.820000,200.000000


In [79]:
df.shape

(930, 11)

In [80]:
df.isnull().sum()

Product Name             0
Actual price             0
Discount price           0
Stars                    0
Rating                   0
Reviews                  0
RAM (GB)               156
Storage (GB)           104
Display Size (inch)      0
Brand                    0
Primary Camera (MP)     73
dtype: int64

#### Add a segment feature

In [81]:
def classify_segment(price):
    if price < 10000:
        return 'Budget'
    elif price <= 30000:
        return 'Mid-Range'
    else:
        return 'Flagship'

df['Segment'] = df['Actual price'].apply(classify_segment)


In [63]:
df.head()

,Product Name,Actual price,Discount price,Stars,Rating,Reviews,RAM (GB),Storage (GB),Display Size (inch),Brand,Primary Camera (MP),Segment
0,"Apple iPhone 15 (Green, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0,Flagship
1,"Apple iPhone 15 (Blue, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0,Flagship
2,"Apple iPhone 15 (Black, 128 GB)",79600,65999,4.6,44793,2402,<NA>,128,6.10,apple,48.0,Flagship
3,"OnePlus N20 SE (JADE WAVE, 128 GB)",19999,11489,4.0,1005,41,4,128,6.56,oneplus,50.0,Mid-Range
4,"OnePlus N20 SE (BLUE OASIS, 64 GB)",16999,12999,4.0,1005,41,4,64,6.56,oneplus,50.0,Mid-Range


In [82]:
# Print unique RAM values
print("Unique RAM (GB) values:")
print(sorted(df['RAM (GB)'].dropna().unique()))

# Print unique Storage values
print("\nUnique Storage (GB) values:")
print(sorted(df['Storage (GB)'].dropna().unique()))

# Print unique Primary Camera values
print("\nUnique Primary Camera (MP) values:")
print(sorted(df['Primary Camera (MP)'].dropna().unique()))

Unique RAM (GB) values:
[np.int64(2), np.int64(3), np.int64(4), np.int64(6), np.int64(8), np.int64(12), np.int64(16), np.int64(32), np.int64(46875)]

Unique Storage (GB) values:
[np.int64(0), np.int64(3), np.int64(4), np.int64(5), np.int64(16), np.int64(32), np.int64(64), np.int64(125), np.int64(128), np.int64(256), np.int64(512)]

Unique Primary Camera (MP) values:
[np.float64(0.0), np.float64(2.0), np.float64(3.0), np.float64(5.0), np.float64(8.0), np.float64(12.0), np.float64(13.0), np.float64(16.0), np.float64(32.0), np.float64(48.0), np.float64(50.0), np.float64(64.0), np.float64(108.0), np.float64(200.0)]


In [83]:
# Standard RAM sizes (GB)
valid_ram = [2, 4, 6, 8, 12, 16, 18, 24] 


def round_to_nearest_valid(value, valid_options):
    if pd.isna(value):
        return value
    # Find the closest valid value
    closest = min(valid_options, key=lambda x: abs(x - value))
    return closest

# Round RAM values
df['RAM (GB)'] = df['RAM (GB)'].apply(lambda x: round_to_nearest_valid(x, valid_ram))

In [84]:
# Print unique RAM values
print("Unique RAM (GB) values:")
print(sorted(df['RAM (GB)'].dropna().unique()))

# Print unique Storage values
print("\nUnique Storage (GB) values:")
print(sorted(df['Storage (GB)'].dropna().unique()))

# Print unique Primary Camera values
print("\nUnique Primary Camera (MP) values:")
print(sorted(df['Primary Camera (MP)'].dropna().unique()))

Unique RAM (GB) values:
[np.float64(2.0), np.float64(4.0), np.float64(6.0), np.float64(8.0), np.float64(12.0), np.float64(16.0), np.float64(24.0)]

Unique Storage (GB) values:
[np.int64(0), np.int64(3), np.int64(4), np.int64(5), np.int64(16), np.int64(32), np.int64(64), np.int64(125), np.int64(128), np.int64(256), np.int64(512)]

Unique Primary Camera (MP) values:
[np.float64(0.0), np.float64(2.0), np.float64(3.0), np.float64(5.0), np.float64(8.0), np.float64(12.0), np.float64(13.0), np.float64(16.0), np.float64(32.0), np.float64(48.0), np.float64(50.0), np.float64(64.0), np.float64(108.0), np.float64(200.0)]


In [85]:
df.shape


(930, 12)

#### Workaround for Null Values

In [86]:
# First, let's define a function to get the mode for a series, handling cases where mode might be empty
def get_mode(series):
    mode_values = series.mode()
    if not mode_values.empty:
        return mode_values.iloc[0]  # Return the first mode if multiple exist
    return None

# For RAM (GB)
df['RAM (GB)'] = df.groupby(['Brand', 'Segment'])['RAM (GB)'].transform(
    lambda x: x.fillna(get_mode(x)) if get_mode(x) is not None else x.fillna(df['RAM (GB)'].mode()[0])
)

# For Storage (GB)
df['Storage (GB)'] = df.groupby(['Brand', 'Segment'])['Storage (GB)'].transform(
    lambda x: x.fillna(get_mode(x)) if get_mode(x) is not None else x.fillna(df['Storage (GB)'].mode()[0])
)

# For Primary Camera (MP)
df['Primary Camera (MP)'] = df.groupby(['Brand', 'Segment'])['Primary Camera (MP)'].transform(
    lambda x: x.fillna(get_mode(x)) if get_mode(x) is not None else x.fillna(df['Primary Camera (MP)'].mode()[0])
)

In [87]:
df.isnull().sum()

Product Name           0
Actual price           0
Discount price         0
Stars                  0
Rating                 0
Reviews                0
RAM (GB)               0
Storage (GB)           0
Display Size (inch)    0
Brand                  0
Primary Camera (MP)    0
Segment                0
dtype: int64

In [88]:
df.head()

,Product Name,Actual price,Discount price,Stars,Rating,Reviews,RAM (GB),Storage (GB),Display Size (inch),Brand,Primary Camera (MP),Segment
0,"Apple iPhone 15 (Green, 128 GB)",79600,65999,4.6,44793,2402,8.0,128,6.10,apple,48.0,Flagship
1,"Apple iPhone 15 (Blue, 128 GB)",79600,65999,4.6,44793,2402,8.0,128,6.10,apple,48.0,Flagship
2,"Apple iPhone 15 (Black, 128 GB)",79600,65999,4.6,44793,2402,8.0,128,6.10,apple,48.0,Flagship
3,"OnePlus N20 SE (JADE WAVE, 128 GB)",19999,11489,4.0,1005,41,4.0,128,6.56,oneplus,50.0,Mid-Range
4,"OnePlus N20 SE (BLUE OASIS, 64 GB)",16999,12999,4.0,1005,41,4.0,64,6.56,oneplus,50.0,Mid-Range


In [89]:
df.shape

(930, 12)

### Save the Dataset

In [90]:
df.to_csv('Cleaned_Mobiles_Dataset.csv')